# **Trabajo Final: Análisis del funcionamiento de IES Siglo 21 (2001-2010)**
Alumno: Lautaro Santos Da Silveira

El siguiente trabajo consta de las estrategias realizadas para realizar el proceso de EDA con los datos brindados, para así poder brindar la información requerida por el cliente. 

## Recursos

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn import linear_model

Mounted at /content/drive


## Correccion de tabla de Alu_Materias 

In [2]:
#Cargo la tabla Alu_Materias que contiene informacion de los alumnos y sus materias cursadas, mostrando notas y fecha de cursado.
df_Alu_Materias= pd.read_csv('/content/drive/MyDrive/Inteligencia Artificial/segundo año/tercer/Practica Profecionalizante 1/tp/TP4/Data/Alu_Materia.csv',sep=';')
df_Alu_Materias.head(3)

,Id_Alumno,Id_Carrera,Id_Materia,Semestre,Año,Fecha_Inscrip,Ctro_Ext,Id_Modalidad,Id_Comision,Nota_1p,Fecha_1p,Nota_2p,Fecha_2p,Nota_TP,Fecha_TP,Nota_Recup,Fecha_Recup,Cuatrimestre
0,0x003014EF4D7D79987F42DE8E3080AEFE,6,115,2,2006,2006-08-16 15:02:00,CO,P,7,11.0,2006-10-02 00:00:00,11.0,2006-11-20 00:00:00,11.0,2006-11-29 00:00:00,11.0,2006-11-27 00:00:00,4
1,0x003014EF4D7D79987F42DE8E3080AEFE,6,121,1,2005,2005-03-10 18:12:00,CO,P,7,11.0,2005-05-04 00:00:00,11.0,2005-06-22 00:00:00,10.0,2005-12-05 15:44:00,11.0,2005-06-28 00:00:00,2
2,0x003014EF4D7D79987F42DE8E3080AEFE,6,121,2,2005,2005-08-17 18:50:00,CO,P,7,11.0,2005-10-05 00:00:00,11.0,2005-11-14 00:00:00,10.0,2005-12-05 15:44:00,11.0,2005-11-21 00:00:00,2


In [3]:
#Observo el tipo de datos de cada columna y si hay datos faltantes
df_Alu_Materias.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131939 entries, 0 to 131938
Data columns (total 18 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Id_Alumno      131939 non-null  object 
 1   Id_Carrera     131939 non-null  int64  
 2   Id_Materia     131939 non-null  int64  
 3   Semestre       131939 non-null  int64  
 4   Año            131939 non-null  int64  
 5   Fecha_Inscrip  131939 non-null  object 
 6   Ctro_Ext       131939 non-null  object 
 7   Id_Modalidad   131939 non-null  object 
 8   Id_Comision    131939 non-null  int64  
 9   Nota_1p        129357 non-null  float64
 10  Fecha_1p       129330 non-null  object 
 11  Nota_2p        110864 non-null  float64
 12  Fecha_2p       110864 non-null  object 
 13  Nota_TP        130064 non-null  float64
 14  Fecha_TP       130064 non-null  object 
 15  Nota_Recup     56441 non-null   float64
 16  Fecha_Recup    56441 non-null   object 
 17  Cuatrimestre   131939 non-nul

In [4]:
#Observo los valores de Ctro_Ext
display(df_Alu_Materias['Ctro_Ext'].value_counts())
#Filtro solo las que tienen informacion de Cordoba, ya que es lo que me interesa analizar
df_Alu_Materias.query("Ctro_Ext=='CO'",inplace=True)

CO    130381
JM       901
VM       206
VA       139
SF       100
CP        90
RC        69
AR        32
HE        13
VD         8
Name: Ctro_Ext, dtype: int64

In [5]:
#Con el .describe puedo observar el año mínimo y máximo, observando así que tengo datos erroneos. 
df_Alu_Materias['Año'].describe()

count    130381.000000
mean       2007.040106
std           3.243244
min        1982.000000
25%        2005.000000
50%        2007.000000
75%        2009.000000
max        2021.000000
Name: Año, dtype: float64

In [6]:
# A los años menores a 2001 y mayores a 2010 los reemplazo por np.nan, ya que puede que solo ese dato esté mal y el resto de información bien.
df_Alu_Materias.loc[(df_Alu_Materias['Año']<2001) | (df_Alu_Materias['Año']>2010),['Fecha_Inscrip','Año']]=np.nan

In [7]:
#Observo que los años se hayan filtrado bien
display(np.sort(df_Alu_Materias['Año'].unique()))
#Observo mis notas posibles para cada materia.
np.sort(df_Alu_Materias['Nota_1p'].unique())

array([2001., 2002., 2003., 2004., 2005., 2006., 2007., 2008., 2009.,
       2010.,   nan])

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., nan])

In [8]:
#Elimino las carreras con ID 59, 41, 40, 11, 12 y 18 ya que contiene muy pocos alumnos
df_Alu_Materias=df_Alu_Materias.loc[~df_Alu_Materias['Id_Carrera'].isin([59,41,40,11,12,18])]

In [9]:
#Al ordenar por Fecha_Inscrip puedo observar que los alumnos que se inscribieron en el año 2000 pero cursen en el 2001 no se hayan eliminado.
df_Alu_Materias.sort_values('Fecha_Inscrip').head(3)

,Id_Alumno,Id_Carrera,Id_Materia,Semestre,Año,Fecha_Inscrip,Ctro_Ext,Id_Modalidad,Id_Comision,Nota_1p,Fecha_1p,Nota_2p,Fecha_2p,Nota_TP,Fecha_TP,Nota_Recup,Fecha_Recup,Cuatrimestre
129969,0x55CE7338DDD5859A4C2E65812B51B590,13,293,1,2001.0,2000-05-31 00:00:00,CO,P,1,9.0,2001-07-05 00:00:00,8.0,2001-07-05 00:00:00,10.0,2005-12-05 15:44:00,10.0,2001-07-05 00:00:00,1
129968,0x55CE7338DDD5859A4C2E65812B51B590,13,259,1,2001.0,2000-05-31 00:00:00,CO,P,1,5.0,2001-07-05 00:00:00,7.0,2001-07-05 00:00:00,10.0,2005-12-05 15:44:00,9.0,2001-07-05 00:00:00,1
129967,0x55CE7338DDD5859A4C2E65812B51B590,13,6,1,2001.0,2000-05-31 00:00:00,CO,P,1,9.0,2001-07-05 00:00:00,8.0,2001-07-05 00:00:00,10.0,2005-12-05 15:44:00,10.0,2001-07-05 00:00:00,2


## Correccion de tabla de Alu_Carrera 

In [10]:
#Cargo la tabla Alu_Carreras, que contiene informacion de los alumnos de la institucion, mostrando su informacion personal e institucional.
df_Alu_Carrera=pd.read_csv('/content/drive/MyDrive/Inteligencia Artificial/segundo año/tercer/Practica Profecionalizante 1/tp/TP4/Data/Alu_Carrera.csv',sep=';')
df_Alu_Carrera.head(3)

,Id_Alumno,Id_Carrera,Semestre_Ingreso,Año_Ingreso,Id_Ctro_Ext,Id_Modalidad,Fecha_Ingreso,Activo,Sexo,Fecha_Nac
0,0x12DA11908B130262F20F714810A49680,6,2,2010,CO,P,2010-07-30 00:00:00,S,M,3/3/1991 00:00
1,0xFCDE4E2EACC469337D296ECE6B5083C6,17,1,2006,CO,P,2006-02-24 00:00:00,S,M,20/11/1954 00:00
2,0xA3BDBF19D2825B585BDC0F65E86D8796,5,2,2002,CO,P,2002-07-08 00:00:00,S,F,24/8/1954 00:00


In [11]:
#Con .info() visualizo el tipo de datos de cada variable
df_Alu_Carrera.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6227 entries, 0 to 6226
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Id_Alumno         6227 non-null   object
 1   Id_Carrera        6227 non-null   int64 
 2   Semestre_Ingreso  6227 non-null   int64 
 3   Año_Ingreso       6227 non-null   int64 
 4   Id_Ctro_Ext       6227 non-null   object
 5   Id_Modalidad      6227 non-null   object
 6   Fecha_Ingreso     6227 non-null   object
 7   Activo            6227 non-null   object
 8   Sexo              6227 non-null   object
 9   Fecha_Nac         6227 non-null   object
dtypes: int64(3), object(7)
memory usage: 486.6+ KB


In [12]:
#Como las fechas están en tipo objeto, las paso a datetime
df_Alu_Carrera['Fecha_Nac']=pd.to_datetime(df_Alu_Carrera['Fecha_Nac'])
df_Alu_Carrera['Fecha_Ingreso']=pd.to_datetime(df_Alu_Carrera['Fecha_Ingreso'])

In [13]:
#Observo los valores de Id_Ctro_Ext
display(df_Alu_Carrera['Id_Ctro_Ext'].value_counts())
#Filtro solo las que tienen informacion de Cordoba, ya que es lo que me interesa analizar
df_Alu_Carrera.query("Id_Ctro_Ext=='CO'",inplace=True)

CO    6215
SF       4
JM       3
VA       2
VM       2
AR       1
Name: Id_Ctro_Ext, dtype: int64

In [14]:
#Calculo la edad de ingreso a la carrera para cada alumno restando la fecha de ingreso con la fecha de nacimiento, obteniendo los días, a esto lo divido 
#en 365.2425, que es la cantidad de días en un año, tomando los decimales como años biciestos, para así obtener la cantidad de años, al final lo paso a numero entero.
x=(df_Alu_Carrera['Fecha_Ingreso']-df_Alu_Carrera['Fecha_Nac']).dt.days
df_Alu_Carrera['Edad_De_Ingreso_A_Carrera']=(x/365.2425).astype(int)
#Elimino la columna Fecha_Nac ya que no me aportará mas información
df_Alu_Carrera.drop(['Fecha_Nac'],axis=1,inplace=True)
df_Alu_Carrera.head(3)

,Id_Alumno,Id_Carrera,Semestre_Ingreso,Año_Ingreso,Id_Ctro_Ext,Id_Modalidad,Fecha_Ingreso,Activo,Sexo,Edad_De_Ingreso_A_Carrera
0,0x12DA11908B130262F20F714810A49680,6,2,2010,CO,P,2010-07-30,S,M,19
1,0xFCDE4E2EACC469337D296ECE6B5083C6,17,1,2006,CO,P,2006-02-24,S,M,51
2,0xA3BDBF19D2825B585BDC0F65E86D8796,5,2,2002,CO,P,2002-07-08,S,F,47


In [15]:
#Observo que tengo edades que estan mal cargadas, ya que solo quiero edades desde los 17 años en adelante, ya que es la edad cuando una persona 
#puede empezar la universidad o estudios terciarios
display(np.sort(df_Alu_Carrera['Edad_De_Ingreso_A_Carrera'].unique()))
#Propongo que las edades menores a 17 tengan valores nulos
df_Alu_Carrera.loc[df_Alu_Carrera['Edad_De_Ingreso_A_Carrera']<17,'Edad_De_Ingreso_A_Carrera']=np.nan
#Observo que se hayan filtrado bien los valores
np.sort(df_Alu_Carrera['Edad_De_Ingreso_A_Carrera'].unique())

array([-18, -11,  -9,  -8,  -7,  -6,  -5,  -4,  -3,   2,  13,  16,  17,
        18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,
        31,  32,  33,  34,  35,  36,  37,  38,  39,  41,  42,  47,  51,
        53])

array([17., 18., 19., 20., 21., 22., 23., 24., 25., 26., 27., 28., 29.,
       30., 31., 32., 33., 34., 35., 36., 37., 38., 39., 41., 42., 47.,
       51., 53., nan])

In [16]:
#Para un mejor manejo de los datos y efectos de visualizacion, genero rangos de edades
bins= [15,25,30,35,40,45,50,100]
name= ['17-25','26-30','31-35','36-40','41-45','46-50','+51']
df_Alu_Carrera['Rang_De_Edad_De_Ingreso_A_Carrera']=pd.cut(df_Alu_Carrera['Edad_De_Ingreso_A_Carrera'],bins=bins,labels=name)
#Elimino la columna Edad_De_Ingreso_A_Carrera ya que los tengo agrupados en otra columna
df_Alu_Carrera.drop(['Edad_De_Ingreso_A_Carrera'],axis=1,inplace=True)

In [17]:
df_Alu_Carrera.sort_values('Fecha_Ingreso').head(3)

,Id_Alumno,Id_Carrera,Semestre_Ingreso,Año_Ingreso,Id_Ctro_Ext,Id_Modalidad,Fecha_Ingreso,Activo,Sexo,Rang_De_Edad_De_Ingreso_A_Carrera
771,0x00C81052F138268CBE003F77E44E1FA6,2,1,2001,CO,P,2000-10-31,S,F,17-25
1175,0x3EF74057900FFDBE3AE264963412628E,4,1,2001,CO,P,2001-01-02,S,F,17-25
1819,0x4191A903B81655C50E18428A74C09AEA,8,1,2001,CO,P,2001-01-03,S,M,17-25


In [18]:
# Observo que carreras puedo eliminar, ya que hay algunas que solo duran un año y tienen pocos integrantes, se los puede tomar como datos molestos.
df_Alu_Carrera['Id_Carrera'].value_counts()

5     962
6     687
7     675
2     641
4     608
1     582
3     506
9     494
13    334
10    270
8     204
17    126
14     57
60     37
16     13
39      8
18      3
11      2
12      2
40      2
41      1
59      1
Name: Id_Carrera, dtype: int64

In [19]:
#Observo que las carreras con ID 59, 41, 40, 11, 12 y 18 tienen pocos alumnos en todo el periodo y ademas duran uno o dos años, por los que se los elimina.
#Esta observacion se aplica para todas las tablas, por lo que en Alu_Materias, Alu_Carreras y Carreras se elimina esta carrera.
df_Alu_Carrera=df_Alu_Carrera.loc[~df_Alu_Carrera['Id_Carrera'].isin([59,41,40,11,12,18])].sort_values('Id_Alumno')

## Tabla de Materias 

In [20]:
#Cargo la tabla de Materias, que contiene el nombre de cada materia. No realizo ningún filtrado ni modificacion porque solo contiene el id de materia y el nombre de materia
Df_Materias= pd.read_csv('/content/drive/MyDrive/Inteligencia Artificial/segundo año/tercer/Practica Profecionalizante 1/tp/TP4/Data/Materias.csv',sep=';')
Df_Materias.head(3)

,Id_Materia,Nombre
0,2,HISTORIA DEL DISEÑO
1,3,INTRODUCCIÓN A LA PUBLICIDAD
2,4,ELEMENTOS DE DISEÑO I


## Tabla de carreras 

In [21]:
#Cargo la tabla Carreras, que contiene informacion de las carreras de la institucion, mostrando sus nombres y la cantidad de cuatrimestres que este contiene.
df_Carreras= pd.read_csv('/content/drive/MyDrive/Inteligencia Artificial/segundo año/tercer/Practica Profecionalizante 1/tp/TP4/Data/Carreras.csv',sep=';')
#Elimino las carreras con ID 59,41,40,11,12,18
df_Carreras=df_Carreras.loc[~df_Carreras['Id_Carrera'].isin([59,41,40,11,12,18])]
df_Carreras.head(3)

,Id_Carrera,Nombre_Largo,Nombre_Corto,Abreviatura,Cuatrimestres
0,9,TECNICO SUPERIOR EN ADMINISTRACION AGROPECUARIA,AGRONEGOCIOS,AA,6
1,3,TECNICO SUPERIOR EN PUBLICIDAD,PUBLICIDAD,PUB,6
3,6,TECNICO SUPERIOR EN RELACIONES PUBLICAS E INST...,RELACIONES PÚBLICAS,RRPP,6


In [22]:
#Para modo de analisis, separo las diferentes carreras con respecto a sus orientaciones, pudiendo ser orientada a Negocios, Comunicación y Diseño,
#Gestión y Tecnología, informacion sacada de la pagina de IES21
Negocios= ['AGRONEGOCIOS','ADMINISTRACIÓN DE EMPRESAS','MULTIMEDIOS','RELACIONES PÚBLICAS','MARKETING','COMERCIO INTERNACIONAL']
Comunicacion_y_Diseño=['PUBLICIDAD','DISEÑO GRÁFICO','ADMINISTRADOR DE REDES']
Gestion=['RECURSOS HUMANOS','TURISMO','ADMINISTRACIÓN DE EMPRESAS','GESTIÓN JURíDICA',
         'GESTIÓN CONTABLE Y FINANCIERA','GESTIÓN AMBIENTAL']
Tecnologias=['ANALISTA DE SISTEMAS','VIDEOJUEGOS']
#Se crea una nueva columna y se completa con su orientación correspondiente.
df_Carreras['Orientacion']=df_Carreras['Nombre_Corto']
df_Carreras.loc[df_Carreras['Orientacion'].isin(Negocios),'Orientacion']='NEGOCIOS'
df_Carreras.loc[df_Carreras['Orientacion'].isin(Comunicacion_y_Diseño),'Orientacion']='COMyDIS'
df_Carreras.loc[df_Carreras['Orientacion'].isin(Gestion),'Orientacion']='GESTION'
df_Carreras.loc[df_Carreras['Orientacion'].isin(Tecnologias),'Orientacion']='TECNOLOGIA'

In [23]:
df_Carreras.head(3)

,Id_Carrera,Nombre_Largo,Nombre_Corto,Abreviatura,Cuatrimestres,Orientacion
0,9,TECNICO SUPERIOR EN ADMINISTRACION AGROPECUARIA,AGRONEGOCIOS,AA,6,NEGOCIOS
1,3,TECNICO SUPERIOR EN PUBLICIDAD,PUBLICIDAD,PUB,6,COMyDIS
3,6,TECNICO SUPERIOR EN RELACIONES PUBLICAS E INST...,RELACIONES PÚBLICAS,RRPP,6,NEGOCIOS


## Matriculación entre 2001 y 2010 

In [24]:
# Unimos la tablas de Alu_Carrera y Carreras para así tratar de unir a la tabla de Alu_Carreras la informacion de la abreviatura, orientacion y numero de cuatrimestres de 
# la carrera
df_Matriculacion= pd.merge(df_Alu_Carrera,df_Carreras)
# Eliminamos las columnas que no me servirán para el análisis
df_Matriculacion.drop(['Id_Carrera','Id_Ctro_Ext','Id_Modalidad','Activo','Nombre_Largo','Nombre_Corto','Fecha_Ingreso'],axis=1,inplace=True)
df_Matriculacion.head(3)

,Id_Alumno,Semestre_Ingreso,Año_Ingreso,Sexo,Rang_De_Edad_De_Ingreso_A_Carrera,Abreviatura,Cuatrimestres,Orientacion
0,0x0002EF737D1D58D4FE5B790A5290753F,2,2001,M,26-30,MKT,6,NEGOCIOS
1,0x00765540B9CA6DBBE92820F203D443C6,1,2003,F,17-25,MKT,6,NEGOCIOS
2,0x00C81052F138268CBE003F77E44E1FA6,1,2001,F,17-25,MKT,6,NEGOCIOS


### Gráficas

A continuación se presentan una serie de gráficas extraidas del DataFrame df_Matriculacion con la informacion de matriculaciones por año, separados por edad y genero, mostrando esta información por Carrera y por Orientacion

#### Cantidad de matriculaciones por año diferenciados en semestres

##### Grafico de lineas Por Semestre de ingreso para cada carrera

In [25]:
#Agrupo la tabla con los datos de la abrebiatura de carrera, año y semestre de ingreso contando la Id de cada alumno
Df_lineas_Carrera=df_Matriculacion.groupby(['Abreviatura','Año_Ingreso','Semestre_Ingreso'])['Id_Alumno'].count().reset_index(name='Cantidad')
Df_lineas_Carrera.head(3)

,Abreviatura,Año_Ingreso,Semestre_Ingreso,Cantidad
0,AA,2001,1,24
1,AA,2001,2,14
2,AA,2002,1,28


In [26]:
#Genero la grafica donde el cuadrante izquierdo representa la cantidad de inscripciones de cada año para el primer semestre, y a la derecha se encuentra
#las inscripciones para el segundo semenstre.
import plotly.express as px
fig = px.line(Df_lineas_Carrera, x="Año_Ingreso", y="Cantidad", color='Abreviatura',facet_col="Semestre_Ingreso")
fig.show()

##### Grafico de lineas Por Semestre de ingreso para cada Orientacion

In [27]:
#Genero un .groupby con los datos de la orientacion de carrera, año y semestre de ingreso contando la Id de cada alumno
Df_lineas_Orientacion=df_Matriculacion.groupby(['Orientacion','Año_Ingreso','Semestre_Ingreso'])['Id_Alumno'].count().reset_index(name='Cantidad')
Df_lineas_Orientacion.head(3)

,Orientacion,Año_Ingreso,Semestre_Ingreso,Cantidad
0,COMyDIS,2001,1,115
1,COMyDIS,2001,2,26
2,COMyDIS,2002,1,95


In [28]:
#Genero la grafica donde cada cuadrante representa una de las orientaciones, como son solo 4 grupos pude realizarlo, a excepcion de la anterior que eran
#demasiados datos para separar.
import plotly.express as px
fig = px.line(Df_lineas_Orientacion, x="Año_Ingreso", y="Cantidad", color='Semestre_Ingreso',facet_col="Orientacion")
fig.show()

Con las anteriores gráficas, se puede ver que la mayoria de matriculaciones a carreras se realizan en el primer semestre, como se esperaba.
     
Otra observacion notable es que las carreras con orientacion a Negocios tiene un mayor numero de matriculaciones. Mientras que las carreras con orientacion a Tecnología terminó con pendiente positiva en matriculaciones, al contrario con las anteriores, el pasaje de 2009 a 2010 terminó con una pendiente negativa, mostrando que las carreras con Orientaciones tecnologicas irán creciendo.

#### Edad de las personas al momento de matricularse

##### proporcion de edades por carrera

In [29]:
#Genero un .groupby con los datos de la Abreviatura de carrera, año y el rango de la edad, contando la Id de cada alumno
Df_Edad_Carrera=df_Matriculacion.groupby(['Abreviatura','Año_Ingreso','Rang_De_Edad_De_Ingreso_A_Carrera'])['Id_Alumno'].count().reset_index(name='Cantidad')
Df_Edad_Carrera.head(3)

,Abreviatura,Año_Ingreso,Rang_De_Edad_De_Ingreso_A_Carrera,Cantidad
0,AA,2001,17-25,35
1,AA,2001,26-30,2
2,AA,2001,31-35,0


In [30]:
#Genero la grafica donde para cada carrera contiene cada barra, y cada barra está separada por la edad de las personas. Los datos se muestran 
#año tras año en la barra inferior
import plotly.express as px
fig = px.bar(Df_Edad_Carrera, x="Abreviatura", y="Cantidad",color="Rang_De_Edad_De_Ingreso_A_Carrera", title="Cantidad de alumnos por Edades y Carrera",
             orientation='v',animation_frame="Año_Ingreso")
fig.show()

##### proporcion de edades por orientacion

In [31]:
#Genero un .groupby con los datos de la orientacion de carrera, año y el rango de la edad, contando la Id de cada alumno
Df_Orientacion_Edad=df_Matriculacion.groupby(['Orientacion','Año_Ingreso','Rang_De_Edad_De_Ingreso_A_Carrera'])['Id_Alumno'].count().reset_index(name='Cantidad')
Df_Orientacion_Edad.head(3)

,Orientacion,Año_Ingreso,Rang_De_Edad_De_Ingreso_A_Carrera,Cantidad
0,COMyDIS,2001,17-25,131
1,COMyDIS,2001,26-30,9
2,COMyDIS,2001,31-35,0


In [32]:
#Genero la grafica donde para cada Orientacion contiene cada barra, y cada barra está separada por la edad de las personas. Los datos se muestran 
#año tras año en la barra inferior
import plotly.express as px
fig = px.bar(Df_Orientacion_Edad, x="Orientacion", y="Cantidad",color="Rang_De_Edad_De_Ingreso_A_Carrera", title="Cantidad de alumnos por Edades y Orientacion",
             orientation='v',animation_frame="Año_Ingreso")
fig.show()

Se puede apreciar notablemente que la mayoria de personas matriculadas pertenece al grupo con edad 17-25, seguidos por los de 26-30 y sucesivamente.Como vimos en el anterior análisis, la Orientacion que mas matriculaciones contiene son las carreras orientadas a Negocios.

Otra observacion notoria es el avance de la carrera de Informatica, en los años anteriores a 2010 tenia una cantidad de matriculaciones moderada, mientras que en el año 2010 se convierte en la carrera con mayor matriculaciones.

Otra observacion que se puede notar es que en las carreras con orientacion de informatica, el nivel de proporcion con respecto a las otras orientaciones, las personas con edades pertenecientes al grupo 26/30 es mas predominante en el último año, mientras que el grupo 17/25 el predominante es la orientación de negocios.

#### Proporcion de genero al momento de matricularse

##### proporcion de genero por carrera

In [33]:
#Genero un .groupby con los datos de la Abreviatura de carrera, año y el sexo, contando la Id de cada alumno
Df_Genero_Carrera=df_Matriculacion.groupby(['Abreviatura',"Año_Ingreso",'Sexo'])['Sexo'].count().reset_index(name='cantidad')
Df_Genero_Carrera.head(3)

,Abreviatura,Año_Ingreso,Sexo,cantidad
0,AA,2001,F,3
1,AA,2001,M,35
2,AA,2002,F,3


In [34]:
#Genero la grafica donde para cada Carrera contiene cada barra, y cada barra está separada por el sexo de los integrantes. Los datos se muestran 
#año tras año en la barra inferior
import plotly.express as px
fig = px.bar(Df_Genero_Carrera, x="Abreviatura", y="cantidad",color="Sexo", title="Proporcion de Genero por Carrera",
             orientation='v',animation_frame="Año_Ingreso")
fig.show()

##### proporcion de genero para orientaciones

In [35]:
#Genero un .groupby con los datos de la Orientacion de carrera, año y el sexo, contando la Id de cada alumno
Df_Genero_Orientacion=df_Matriculacion.groupby(['Orientacion',"Año_Ingreso",'Sexo'])['Sexo'].count().reset_index(name='cantidad')
Df_Genero_Orientacion.head(3)

,Orientacion,Año_Ingreso,Sexo,cantidad
0,COMyDIS,2001,F,70
1,COMyDIS,2001,M,71
2,COMyDIS,2002,F,53


In [36]:
#Genero la grafica donde para cada Orientacion contiene cada barra, y cada barra está separada por el sexo de los integrantes. Los datos se muestran 
#año tras año en la barra inferior
import plotly.express as px
fig = px.bar(Df_Genero_Orientacion, x="Orientacion", y="cantidad",color="Sexo", title="Proporcion de Genero por orientacion",
             orientation='v',animation_frame="Año_Ingreso")
fig.show()

Gracias a la grafica que separa por Orientacion, podemos apreciar las diferencias de género por cada una, siendo la unica especialidad mas igualada de valores en la de comunicación y diseño, puesto que gestión, la mayoria de matriculaciones son por parte de mujeres, al contrario ocurre con negocios y tecnología

## Condicion regular/libre/promocionado 

In [37]:
#Genero una copia de df_Alu_Materias para trabajar y si ocasiono un error no me generaría problemas en la tabla para futuros análisis
df_Alumnos_Materia=df_Alu_Materias.copy()
df_Alumnos_Materia.head(3)

,Id_Alumno,Id_Carrera,Id_Materia,Semestre,Año,Fecha_Inscrip,Ctro_Ext,Id_Modalidad,Id_Comision,Nota_1p,Fecha_1p,Nota_2p,Fecha_2p,Nota_TP,Fecha_TP,Nota_Recup,Fecha_Recup,Cuatrimestre
0,0x003014EF4D7D79987F42DE8E3080AEFE,6,115,2,2006.0,2006-08-16 15:02:00,CO,P,7,11.0,2006-10-02 00:00:00,11.0,2006-11-20 00:00:00,11.0,2006-11-29 00:00:00,11.0,2006-11-27 00:00:00,4
1,0x003014EF4D7D79987F42DE8E3080AEFE,6,121,1,2005.0,2005-03-10 18:12:00,CO,P,7,11.0,2005-05-04 00:00:00,11.0,2005-06-22 00:00:00,10.0,2005-12-05 15:44:00,11.0,2005-06-28 00:00:00,2
2,0x003014EF4D7D79987F42DE8E3080AEFE,6,121,2,2005.0,2005-08-17 18:50:00,CO,P,7,11.0,2005-10-05 00:00:00,11.0,2005-11-14 00:00:00,10.0,2005-12-05 15:44:00,11.0,2005-11-21 00:00:00,2


In [38]:
#Elimino las variables que no necesito para el análisis, obteniendo así mayor comodidad para la visualización de los datos 
df_Alumnos_Materia.drop(['Fecha_Inscrip','Ctro_Ext','Id_Modalidad','Id_Comision','Id_Comision','Fecha_1p','Fecha_2p','Fecha_TP','Fecha_Recup'],axis=1,inplace=True)
df_Alumnos_Materia.head(3)

,Id_Alumno,Id_Carrera,Id_Materia,Semestre,Año,Nota_1p,Nota_2p,Nota_TP,Nota_Recup,Cuatrimestre
0,0x003014EF4D7D79987F42DE8E3080AEFE,6,115,2,2006.0,11.0,11.0,11.0,11.0,4
1,0x003014EF4D7D79987F42DE8E3080AEFE,6,121,1,2005.0,11.0,11.0,10.0,11.0,2
2,0x003014EF4D7D79987F42DE8E3080AEFE,6,121,2,2005.0,11.0,11.0,10.0,11.0,2


In [39]:
#Para saber si un alumno está promocionado en una materia, todas sus notas deben ser superiores a 7 
df_Alumnos_Materia.loc[(df_Alumnos_Materia['Nota_1p']>=7) & (df_Alumnos_Materia['Nota_1p']<11) & 
                                 (df_Alumnos_Materia['Nota_2p']>=7) & (df_Alumnos_Materia['Nota_2p']<11) &
                                 (df_Alumnos_Materia['Nota_TP']>=7) & (df_Alumnos_Materia['Nota_TP']<11),
                                 'Condicion']='Promocionado'
#Para establecer que un alumno está libre, sus notas deben ser menores a 4 o igual a 11 en cualquiera de las cuatro,
# siendo esta condición de ausencia. Ademas, si la nota de TP es menor a 4 o 11, automaticamente se lo considera como libre
df_Alumnos_Materia.loc[((df_Alumnos_Materia['Nota_TP']<4) | (df_Alumnos_Materia['Nota_TP']==11))|
                       ((((df_Alumnos_Materia['Nota_1p']<4) | (df_Alumnos_Materia['Nota_1p']==11)) | 
                       ((df_Alumnos_Materia['Nota_2p']<4) | (df_Alumnos_Materia['Nota_2p']==11))) &
                       ((df_Alumnos_Materia['Nota_Recup']<4) | (df_Alumnos_Materia['Nota_Recup']==11))),
                       'Condicion']='Libre'

#Las demas condiciones las considero como regular 
df_Alumnos_Materia['Condicion'].replace(np.nan,'Regular',inplace=True)

In [40]:
# Como algunos valores tenian el año sin datos, los tomaba como si fuesen desertores, por eso reemplacé los valores solo de Desertores, ya que egresado solo me toma como 
# parametro la cantidad de cuatrimestres cursado.
df_Alumnos_Materia.loc[(df_Alumnos_Materia['Nota_1p'].isna()) & (df_Alumnos_Materia['Nota_2p'].isna()) &
                       (df_Alumnos_Materia['Nota_TP'].isna()) & (df_Alumnos_Materia['Nota_Recup'].isna()),'Condicion'] = df_Alumnos_Materia['Condicion'].replace('Regular',np.nan)

In [41]:
df_Alumnos_Materia.sort_values(['Nota_1p','Nota_2p','Nota_TP'],ascending=True)

,Id_Alumno,Id_Carrera,Id_Materia,Semestre,Año,Nota_1p,Nota_2p,Nota_TP,Nota_Recup,Cuatrimestre,Condicion
11603,0xF20A489FC10724EB1404F6EDC0FB3329,2,37,2,2002.0,0.0,1.0,10.0,5.0,3,Regular
34044,0x5C73E4B45EBDD5B7D13015FC512E2316,4,207,2,2002.0,0.0,1.0,10.0,6.0,4,Regular
10514,0x3E6B517C2D3C881F3BF4739ACB60B246,5,259,2,2001.0,0.0,2.0,10.0,2.0,1,Libre
101912,0xA9F8E38894BA80A647D9772A94CA38BF,1,170,1,NaN,0.0,3.0,5.0,1.0,4,Libre
76616,0x5EAC266BB50DC6ECAC8B193BAD76A3FA,6,91,2,2007.0,0.0,5.0,5.0,2.0,2,Libre
...,...,...,...,...,...,...,...,...,...,...,...
131853,0x48B8C654BA471EF7C2F6442C50449723,13,232,2,2010.0,NaN,NaN,NaN,NaN,6,NaN
131868,0x48B8C654BA471EF7C2F6442C50449723,13,305,2,2008.0,NaN,NaN,NaN,NaN,6,NaN
131873,0x48B8C654BA471EF7C2F6442C50449723,13,500,2,2008.0,NaN,NaN,NaN,NaN,5,NaN
131875,0x48B8C654BA471EF7C2F6442C50449723,13,505,1,2008.0,NaN,NaN,NaN,NaN,6,NaN


### Gráfica

A continuación se presentan una serie de gráficas extraidas del DataFrame df_Alumnos_Materia para mostrar que materias tienen mayor numero de alumnos en las diferentes condiciones academicas (libre/regular/promocionado)

Para facilitar la lectura de la grafica, se unen las tablas de Materias y Carreras, para así observar en que Carreras y Materias hay mayores cantidades de cada uno.

In [42]:
Df_Libre_Promocionado_Regular=pd.merge(df_Alumnos_Materia,Df_Materias)
Df_Libre_Promocionado_Regular=pd.merge(Df_Libre_Promocionado_Regular,df_Carreras)

In [43]:
Df_Libre_Promocionado_Regular.head(3)

,Id_Alumno,Id_Carrera,Id_Materia,Semestre,Año,Nota_1p,Nota_2p,Nota_TP,Nota_Recup,Cuatrimestre,Condicion,Nombre,Nombre_Largo,Nombre_Corto,Abreviatura,Cuatrimestres,Orientacion
0,0x003014EF4D7D79987F42DE8E3080AEFE,6,115,2,2006.0,11.0,11.0,11.0,11.0,4,Libre,TEORÍA Y PRACT. DE LAS RELACIONES PÚBLICAS E I...,TECNICO SUPERIOR EN RELACIONES PUBLICAS E INST...,RELACIONES PÚBLICAS,RRPP,6,NEGOCIOS
1,0xCC6485603EC63ABD87DF858E520C8864,6,115,2,2002.0,7.0,7.0,10.0,NaN,4,Promocionado,TEORÍA Y PRACT. DE LAS RELACIONES PÚBLICAS E I...,TECNICO SUPERIOR EN RELACIONES PUBLICAS E INST...,RELACIONES PÚBLICAS,RRPP,6,NEGOCIOS
2,0xCC6485603EC63ABD87DF858E520C8864,6,115,2,2005.0,7.0,8.0,10.0,NaN,4,Promocionado,TEORÍA Y PRACT. DE LAS RELACIONES PÚBLICAS E I...,TECNICO SUPERIOR EN RELACIONES PUBLICAS E INST...,RELACIONES PÚBLICAS,RRPP,6,NEGOCIOS


In [44]:
#Agrupo el DataFrame obtenido por año, abreviatura de la carrera, nombre de materia y condicion de finalizacion contando cada condicion por materia de carrera.
Grupo=Df_Libre_Promocionado_Regular.groupby(['Año','Abreviatura','Nombre','Condicion'])['Condicion'].count().reset_index(name='Cant')
Grupo.head(3)

,Año,Abreviatura,Nombre,Condicion,Cant
0,2001.0,AA,BIOLOGÍA Y MEDIO AMBIENTE,Libre,2
1,2001.0,AA,BIOLOGÍA Y MEDIO AMBIENTE,Promocionado,19
2,2001.0,AA,BIOLOGÍA Y MEDIO AMBIENTE,Regular,14


#### Condiciones academicas por carrera año tras año ✅

##### Graficos de linea por carrera con valores de condiciones academicas

In [45]:
#Genero un grupo con los datos de la abreviatura, año, condicion academica y cantidad de cada una.
Grafico_linea_condicion_carrera=Df_Libre_Promocionado_Regular.groupby(['Año','Abreviatura','Condicion'])['Condicion'].count().reset_index(name='Cant')
Grafico_linea_condicion_carrera.head(3)

,Año,Abreviatura,Condicion,Cant
0,2001.0,AA,Libre,46
1,2001.0,AA,Promocionado,96
2,2001.0,AA,Regular,145


In [46]:
#Genero la grafica 
import plotly.express as px
fig = px.line(Grafico_linea_condicion_carrera, x="Año", y="Cant", color='Abreviatura',facet_col="Condicion")
fig.show()

##### Graficos de linea por Orientacion con valores de condiciones academicas

In [47]:
#Genero un grupo con los datos de la Orientacion, año, condicion academica y cantidad de cada uno.
Grafico_linea_condicion_Orientacion=Df_Libre_Promocionado_Regular.groupby(['Año','Orientacion','Condicion'])['Condicion'].count().reset_index(name='Cant')
Grafico_linea_condicion_Orientacion.head(3)

,Año,Orientacion,Condicion,Cant
0,2001.0,COMyDIS,Libre,114
1,2001.0,COMyDIS,Promocionado,416
2,2001.0,COMyDIS,Regular,468


In [48]:
#Genero la grafica 
import plotly.express as px
fig = px.line(Grafico_linea_condicion_Orientacion, x="Año", y="Cant", color='Orientacion',facet_col="Condicion")
fig.show()

En las dos graficas se puede observar que hay un mayor numero de alumnos regulares, mientras que los alumnos promocionados se encuentran levemente mas arriba de los libres, pero no en todos los casos. Uno de los casos en que los alumnos con condicion libre superan a los alumnos promocionados ocurre en informatica en el año 2010


#### informacion de condicion academica de los alumnos a lo largo del período 2001-2010 ✅

In [49]:
#Genero un histograma por cada carrera con las cantidades de cada condicion en el período 2001-2010
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Histogram(x=Grupo['Abreviatura'], y=Grupo['Cant'][Grupo['Condicion']=='Libre'],
                name='Libre',
                marker_color='rgb(55, 83, 109)'
                ))
fig.add_trace(go.Histogram(x=Grupo['Abreviatura'], y=Grupo['Cant'][Grupo['Condicion']=='Regular'],
                name='Regular',
                marker_color='rgb(26, 118, 255)'
                ))
fig.add_trace(go.Histogram(x=Grupo['Abreviatura'], y=Grupo['Cant'][Grupo['Condicion']=='Promocionado'],
                name='promociono',
                marker_color='rgb(26, 118, 100)'
                ))
fig.update_layout(
    title='condiciones de alumnos por carrera entre 2001 y 2010',
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='Cantidad',
        titlefont_size=16,
        tickfont_size=14,
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1
)
fig.show()

Se puede apreciar que en todo el perído 2001-2010 la cantidad total de alumnos libres es menor en la mayoria de las carreras que la cantidad de alumnos regulares, lo mismo ocurre con la cantidad de alumnos promocionados pero la diferencia no es tan grande

#### Materias con mayor cantidad de alumnos Promocionados, Regulares y libres por carrera

In [50]:
#Genero un grupo que me traiga la maxima cantidad de libres, regulares y promocionados de cada carrera y materia 
grupo_prueba= pd.DataFrame(Grupo.groupby(['Año','Abreviatura','Condicion'])['Cant'].max()).reset_index()
grupo_prueba.head(3)

,Año,Abreviatura,Condicion,Cant
0,2001.0,AA,Libre,8
1,2001.0,AA,Promocionado,19
2,2001.0,AA,Regular,19


In [51]:
#genero una tabla con los datos de grupo para así obtener el nombre de la materia 
df_condiciones= pd.merge(grupo_prueba,Grupo,how='left')
df_condiciones

,Año,Abreviatura,Condicion,Cant,Nombre
0,2001.0,AA,Libre,8,GEOGRAFÍA ECONÓMICA
1,2001.0,AA,Libre,8,SISTEMAS DE INFORMACIÓN
2,2001.0,AA,Promocionado,19,BIOLOGÍA Y MEDIO AMBIENTE
3,2001.0,AA,Regular,19,RECURSOS INFORMÁTICOS
4,2001.0,AA,Regular,19,SISTEMAS DE INFORMACIÓN
...,...,...,...,...,...
549,2010.0,SVIRT,Regular,17,TÉCNICAS DE DIBUJO
550,2010.0,TUR,Libre,15,FUNDAMENTOS DE ECONOMÍA
551,2010.0,TUR,Libre,15,MERCADOTECNIA
552,2010.0,TUR,Promocionado,12,ADMINISTRACIÓN HOTELERA


In [90]:
#Realizo un grafico de barras para ver año tras año que materias son las que mayormente tienen alumnos libres en cada carrera
import plotly.express as px
fig = px.bar(df_condiciones, x="Abreviatura", y="Cant",color="Nombre", title="Materia que por Carrera tiene mas alumnos libres",
             orientation='v',animation_frame="Año",facet_row='Condicion')
fig.show()

## Condicion de egresado/cursando/desertor 

In [53]:
#Para trabajar con mayor comodidad, genero una copia de Df_Libre_Promocionado_Regular, ya que tiene los datos que necesito y si genero un cambio erroneo no tengo que 
#volver a ejecutar la creacion del DF
df_Alumnos_Materia_ECD=Df_Libre_Promocionado_Regular.copy()
df_Alumnos_Materia_ECD.head(3)

,Id_Alumno,Id_Carrera,Id_Materia,Semestre,Año,Nota_1p,Nota_2p,Nota_TP,Nota_Recup,Cuatrimestre,Condicion,Nombre,Nombre_Largo,Nombre_Corto,Abreviatura,Cuatrimestres,Orientacion
0,0x003014EF4D7D79987F42DE8E3080AEFE,6,115,2,2006.0,11.0,11.0,11.0,11.0,4,Libre,TEORÍA Y PRACT. DE LAS RELACIONES PÚBLICAS E I...,TECNICO SUPERIOR EN RELACIONES PUBLICAS E INST...,RELACIONES PÚBLICAS,RRPP,6,NEGOCIOS
1,0xCC6485603EC63ABD87DF858E520C8864,6,115,2,2002.0,7.0,7.0,10.0,NaN,4,Promocionado,TEORÍA Y PRACT. DE LAS RELACIONES PÚBLICAS E I...,TECNICO SUPERIOR EN RELACIONES PUBLICAS E INST...,RELACIONES PÚBLICAS,RRPP,6,NEGOCIOS
2,0xCC6485603EC63ABD87DF858E520C8864,6,115,2,2005.0,7.0,8.0,10.0,NaN,4,Promocionado,TEORÍA Y PRACT. DE LAS RELACIONES PÚBLICAS E I...,TECNICO SUPERIOR EN RELACIONES PUBLICAS E INST...,RELACIONES PÚBLICAS,RRPP,6,NEGOCIOS


In [54]:
#Genero un grupo de df_Alumnos_Materia_ECD mostrando Id_Alumno, Abreviatura, Orientacion, Semestre, Año y cuatrimestre maximo cursado, y la cantidad de cuatrimestres por carrera.
df_Cuatrimestre_max=df_Alumnos_Materia_ECD.groupby(['Id_Alumno','Abreviatura','Orientacion'])['Semestre','Año','Cuatrimestre','Cuatrimestres'].max().reset_index()
df_Cuatrimestre_max.head(3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,Id_Alumno,Abreviatura,Orientacion,Semestre,Año,Cuatrimestre,Cuatrimestres
0,0x0002EF737D1D58D4FE5B790A5290753F,MKT,NEGOCIOS,2,2006.0,6,6
1,0x00140BF821D6CE41168C9ABBF9056A60,PUB,COMyDIS,2,2010.0,6,6
2,0x00146A709B7887BBB125014B073D11D9,INF,TECNOLOGIA,2,2010.0,4,6


Para determinar si un alumno es Egresado, si está cursando o es un desertor realizo el siguiente código

In [55]:
#Si el alumno alcanzó el maximo de cuatrimestres cursados, es egresado
df_Cuatrimestre_max.loc[(df_Cuatrimestre_max['Cuatrimestre']==df_Cuatrimestre_max['Cuatrimestres']),'Estado']='Egresado'
#Si el alumno no alcanzó el maximo cuatrimestre de la carrera, pero su ultimo dato consta del año 2010, semestre 2 y no es un egresado, el alumno está cursando
df_Cuatrimestre_max.loc[(df_Cuatrimestre_max['Estado']!='Egresado') &
                        (df_Cuatrimestre_max['Cuatrimestre']!=df_Cuatrimestre_max['Cuatrimestres'])&
                        (df_Cuatrimestre_max['Año']==2010)&
                        (df_Cuatrimestre_max['Semestre']==2),'Estado']='Cursando'
#Si el alumno no alcanzó el maximo cuatrimestre de la carrera, pero su ultimo dato es diferente al año 2010, o del año 2010 semestre 1,
# y no es un egresado ni un cursante, el alumno es desertor
df_Cuatrimestre_max.loc[(df_Cuatrimestre_max['Estado']!='Egresado') &
                        (df_Cuatrimestre_max['Estado']!='Cursando') &
                        (df_Cuatrimestre_max['Cuatrimestre']!=df_Cuatrimestre_max['Cuatrimestres'])&
                        (((df_Cuatrimestre_max['Año']==2010)&(df_Cuatrimestre_max['Semestre']==1))|
                         ((df_Cuatrimestre_max['Año']!=2010))),'Estado']='Desertor'

In [56]:
# Como algunos valores tenian el año sin datos, los tomaba como si fuesen desertores, por eso reemplacé los valores solo de Desertores, ya que egresado solo me toma como 
# parametro la cantidad de cuatrimestres cursado.
df_Cuatrimestre_max.loc[(df_Cuatrimestre_max['Año'].isna()) & (df_Cuatrimestre_max['Estado']=='Desertor'),'Estado'] = df_Cuatrimestre_max['Estado'].replace('Desertor',np.nan)

In [57]:
df_Cuatrimestre_max.sort_values('Año')

,Id_Alumno,Abreviatura,Orientacion,Semestre,Año,Cuatrimestre,Cuatrimestres,Estado
4926,0xCBE444723B5394666A856AAAC8E153DE,ADMIN,NEGOCIOS,2,2001.0,2,6,Desertor
228,0x09766AB8202EF64DAC5A00CBD3CE90FE,MM,NEGOCIOS,2,2001.0,2,6,Desertor
892,0x23C91057A2B85CF1A7FBBE5A5CA33128,MM,NEGOCIOS,1,2001.0,3,6,Desertor
4824,0xC73F168141053C40FD3828BA6E652E4C,GA,GESTION,2,2001.0,2,6,Desertor
1540,0x3F0A57CFDAEFBAC35280E91371B427E1,MKT,NEGOCIOS,2,2001.0,2,6,Desertor
...,...,...,...,...,...,...,...,...
5494,0xE367D7D452105F5511F001D9490D87FB,AA,NEGOCIOS,2,NaN,6,6,Egresado
5737,0xED4D760A18031BBE359BF30968D43611,AA,NEGOCIOS,1,NaN,2,6,NaN
5888,0xF39DCC2CA08FAA28EF7B3AB4ED176B0B,INF,TECNOLOGIA,2,NaN,4,6,NaN
6064,0xFA82917C7827CB8458722AC5A35E9E8F,INF,TECNOLOGIA,1,NaN,1,6,NaN


### Gráfica

#### Evolucion año tras año de alumnos desertores y egresados ✅

Tomaré como principal analisis los datos de alumnos desertores y egresados, ya que los que se encuentran cursando solo están en el año 2010

##### Grafico de lineas por Estado de alumno en cada carrera

In [58]:
# Genero un grupo del DF df_Cuatrimestre_max mostrando abreviatura, año y estado contando la cantidad de desertores, egresados o cursantes en todo el periodo por carreras.
Estado= df_Cuatrimestre_max.groupby(['Abreviatura','Año','Estado'])['Id_Alumno'].count().reset_index(name='Cantidad')
# Para un análisis mas preciso, ya que solo en el año 2010 obtendré datos de los alumnos cursantes, solo analizaré las condiciones de Egresados y Desertores
Estado=Estado.loc[Estado['Estado']!='Cursando']
Estado.head(3)

,Abreviatura,Año,Estado,Cantidad
0,AA,2001.0,Desertor,3
1,AA,2002.0,Desertor,10
2,AA,2002.0,Egresado,1


In [59]:
#Genero el gráfico 
import plotly.express as px
fig = px.line(Estado, x="Año", y="Cantidad", color='Abreviatura',facet_col="Estado")
fig.show()

##### Grafico de lineas por Estado de alumno en cada Orientacion

In [60]:
# Genero un grupo del DF df_Cuatrimestre_max mostrando Orientacion, año y estado contando la cantidad de desertores, egresados o cursantes en todo el periodo por carreras.
Estado_orientacion= df_Cuatrimestre_max.groupby(['Orientacion','Año','Estado'])['Id_Alumno'].count().reset_index(name='Cantidad')
# Para un análisis mas preciso, ya que solo en el año 2010 obtendré datos de los alumnos cursantes, solo analizaré las condiciones de Egresados y Desertores
Estado_orientacion=Estado_orientacion.loc[Estado_orientacion['Estado']!='Cursando']
Estado_orientacion.head(3)

,Orientacion,Año,Estado,Cantidad
0,COMyDIS,2001.0,Desertor,29
1,COMyDIS,2001.0,Egresado,2
2,COMyDIS,2002.0,Desertor,36


In [61]:
#Genero el gráfico 
import plotly.express as px
fig = px.line(Estado_orientacion, x="Año", y="Cantidad", color='Orientacion',facet_col="Estado")
fig.show()

A simple vista, se puede observar que año tras año, era mayor el numero de alumnos desertores en comparacion con los Egresados, esta tendencia cambia en el transcurso del año 2009 al 2010, puesto que la cantidad de egresados aumenta, superando notablemente a la cantidad de desertores.

#### Cantidades de alumnos Egresados/Desertores ✅

Si bien las siguientes graficas y la de arriba tienen casi la misma información, de esta se pueden observar mas facilmente los valores en un año en particular, mientras que en la anterior se puede ver la tendencia en todo el período.

##### Grafico de barras en proporcion de cantidad de alumnos egresados/desertores por carrera en cada año

In [62]:
# Genero un grupo del DF df_Cuatrimestre_max mostrando abreviatura, año y estado contando la cantidad de desertores, egresados o cursantes en todo el periodo por carreras.
Estado_Barra_Carrera= df_Cuatrimestre_max.groupby(['Abreviatura','Año','Estado'])['Id_Alumno'].count().reset_index(name='Cantidad')
# Para un análisis mas preciso, ya que solo en el año 2010 obtendré datos de los alumnos cursantes, solo analizaré las condiciones de Egresados y Desertores
Estado_Barra_Carrera=Estado_Barra_Carrera.loc[Estado_Barra_Carrera['Estado']!='Cursando']
Estado_Barra_Carrera.head(3)

,Abreviatura,Año,Estado,Cantidad
0,AA,2001.0,Desertor,3
1,AA,2002.0,Desertor,10
2,AA,2002.0,Egresado,1


In [91]:
# Genero el gráfico
import plotly.express as px
fig = px.bar(Estado_Barra_Carrera, x="Abreviatura", y="Cantidad",color="Estado", title="Cantidad de alumnos por Edades y Carrera",
             orientation='v',animation_frame="Año")
fig.show()

##### Grafico de barras en proporcion de cantidad de alumnos egresados/desertores por Orientacion en cada año

In [64]:
# Genero un grupo del DF df_Cuatrimestre_max mostrando Orientacion, año y estado contando la cantidad de desertores, egresados o cursantes en todo el periodo por carreras.
Estado_Barra_Orientacion= df_Cuatrimestre_max.groupby(['Orientacion','Año','Estado'])['Id_Alumno'].count().reset_index(name='Cantidad')
# Para un análisis mas preciso, ya que solo en el año 2010 obtendré datos de los alumnos cursantes, solo analizaré las condiciones de Egresados y Desertores
Estado_Barra_Orientacion=Estado_Barra_Orientacion.loc[Estado_Barra_Orientacion['Estado']!='Cursando']
Estado_Barra_Orientacion.head(3)

,Orientacion,Año,Estado,Cantidad
0,COMyDIS,2001.0,Desertor,29
1,COMyDIS,2001.0,Egresado,2
2,COMyDIS,2002.0,Desertor,36


In [65]:
#Genero el gráfico 
import plotly.express as px
fig = px.bar(Estado_Barra_Orientacion, x="Orientacion", y="Cantidad",color="Estado", title="Cantidad de alumnos por Edades y Carrera",
             orientation='v',animation_frame="Año")
fig.show()

Como dijimos anteriormente, se puede observar el aumento de egresados en el pasaje del año 2009 al 2010

#### Condicion de alumnos en el año 2010 (tomando a los alumnos que todavia estan cursando) ✅

Como dijimos anteriormente, solo en el año 2010 tenemos la información de los alumnos que todavía estan cursando, por ende añora solo traeremos los datos de ese año y analizaremos la cantidad de alumnos Egresados/Cursantes/Desertores, ademas se podrá visualizar en que carreras hay mayor cantidad de alumnos cursantes

##### Por carrera

In [66]:
# Genero un grupo del DF df_Cuatrimestre_max mostrando abreviatura, año y estado contando la cantidad de desertores, egresados o cursantes en todo el periodo por carreras.
AlumnosCursantes= df_Cuatrimestre_max.groupby(['Abreviatura','Año','Estado'])['Id_Alumno'].count().reset_index(name='Cantidad')
#Traigo solo los datos del año 2010
AlumnosCursantes=AlumnosCursantes.loc[AlumnosCursantes['Año']==2010]
AlumnosCursantes.head(3)

,Abreviatura,Año,Estado,Cantidad
17,AA,2010.0,Cursando,62
18,AA,2010.0,Desertor,13
19,AA,2010.0,Egresado,103


In [67]:
#Genero el grafico
import plotly.express as px
fig = px.bar(AlumnosCursantes, x="Abreviatura", y="Cantidad",
             color='Estado', barmode='group',
             height=400)
fig.show()

##### Por orientacion

In [68]:
# Genero un grupo del DF df_Cuatrimestre_max mostrando Orientacion, año y estado contando la cantidad de desertores, egresados o cursantes en todo el periodo por carreras.
AlumnosCursantes_Orientacion= df_Cuatrimestre_max.groupby(['Orientacion','Año','Estado'])['Id_Alumno'].count().reset_index(name='Cantidad')
#Traigo solo los datos del año 2010
AlumnosCursantes_Orientacion=AlumnosCursantes_Orientacion.loc[AlumnosCursantes_Orientacion['Año']==2010]
AlumnosCursantes_Orientacion.head(3)

,Orientacion,Año,Estado,Cantidad
18,COMyDIS,2010.0,Cursando,113
19,COMyDIS,2010.0,Desertor,23
20,COMyDIS,2010.0,Egresado,368


In [69]:
#Genero el grafico
import plotly.express as px
fig = px.bar(AlumnosCursantes_Orientacion, x="Orientacion", y="Cantidad",
             color='Estado', barmode='group',
             height=400)
fig.show()

De los anteriores gráficos se puede observar que si bien en el año 2010 hubo mayor cantidad de alumnos en la carrera de informática, todavia la orientacion predominante es la destinada al area de negocios.

## Pronosticos futuros

####Cantidad total de desertores esperados en el año 2011

In [70]:
#Genero una copia del DF df_Cuatrimestre_max para así poder modificarlo y que no hayan futuros problemas
df_Pronostico=df_Cuatrimestre_max.copy()
#Filtro para que aparezcan solo las filas que tienen en la variable Estado como Desertor
df_Pronostico=df_Pronostico.loc[df_Pronostico['Estado']=='Desertor']
#Uno el df obtenido con la tabla de carreras para así obtener el id de carrera
df_Pronostico=df_Pronostico.merge(df_Carreras)
df_Pronostico.drop(['Nombre_Largo','Nombre_Corto','Abreviatura','Orientacion','Cuatrimestres','Cuatrimestre'],axis=1,inplace=True)
df_Pronostico

,Id_Alumno,Semestre,Año,Estado,Id_Carrera
0,0x0027188BBD44242F5A3F2C9592576A15,2,2009.0,Desertor,6
1,0x003014EF4D7D79987F42DE8E3080AEFE,2,2006.0,Desertor,6
2,0x0141F8A2AA272331032B3F2507BAD59E,1,2008.0,Desertor,6
3,0x01A769D05D4F9DC6FDCCEA01CEF12B88,2,2005.0,Desertor,6
4,0x020F23967823F926274678A3780B91D8,2,2008.0,Desertor,6
...,...,...,...,...,...
2526,0x26240E224B6C59A147F450DBCB7B62C9,2,2005.0,Desertor,39
2527,0x4F8CDCC18445B66272EAD975E0DE6768,2,2006.0,Desertor,39
2528,0xBBC26273A1E875EBA9696B72C1A6AC1D,2,2005.0,Desertor,39
2529,0xEB424B36C0C3BD9BB5BE43C42F197B6D,1,2005.0,Desertor,39


##### En la totalidad de la institución

In [92]:
#Genero un groupby por año contando la cantidad de desertores que hay en toda la institucion
grupo_1=df_Pronostico.groupby(['Año','Semestre'])['Estado'].count().reset_index(name='cant')
x= grupo_1.drop(['cant'],axis=1)
y= grupo_1['cant']
#Genero un modelo de Regresion Lineal, al ser pocos datos, no genero los sets de train y test
from sklearn import linear_model
modelo = linear_model.LinearRegression(fit_intercept=True)
modelo.fit(x,y)
#El valor de 'y' es la cantidad de alumnos desertores para el año 2011, multiplicando 2011 * .coef_[0], que en una ecuacion de recta sería igual al valor de la pendiente,
# mas el numero de semestre correspondiente * su coeficiente correspondiente y le sumo el .intercept_ que es el valor de la ordenada de origen
y_1=2011*(modelo.coef_[0]) + 1*(modelo.coef_[1])+modelo.intercept_
y_2=2011*(modelo.coef_[0]) + 2*(modelo.coef_[1])+modelo.intercept_
#El siguiente codigo pone al valor de y_1 o y_2 como 0 si este es menor o igual a 0, ademas de redondear el valor a numero entero.
if y_1<=0:
  y_1=0
else:
  y_1=y_1.round()
if y_2<=0:
  y_2=0
else:
  y_2=y_2.round()
print(f'Se espera que en el año 2011 haya un total de {y_1.round()} Desertores en el periodo y {y_2.round()} para el segundo semestre')

Se espera que en el año 2011 haya un total de 162.0 Desertores en el periodo y 271.0 para el segundo semestre


In [93]:
grupo_1

,Año,Semestre,cant
0,2001.0,1,82
1,2001.0,2,90
2,2002.0,1,44
3,2002.0,2,126
4,2003.0,1,42
5,2003.0,2,138
6,2004.0,1,51
7,2004.0,2,195
8,2005.0,1,72
9,2005.0,2,161


##### Por carrera

In [94]:
#El siguiente código es parecido al anterior, solo que se le añade un imput que genera el codigo de id de carrera para analizar datos específicos
grupo_2=df_Pronostico.groupby(['Año','Id_Carrera','Semestre'])['Estado'].count().reset_index(name='cant')
codigo= int(input('Ingrese el codigo de la carrera: '))
grupo_2=grupo_2[grupo_2['Id_Carrera']==codigo]
x= grupo_2.drop(['cant'],axis=1)
y= grupo_2['cant']
#Genero un modelo de Regresion Lineal, al ser pocos datos, no genero los sets de train y test
from sklearn import linear_model
modelo = linear_model.LinearRegression(fit_intercept=True)
modelo.fit(x,y)
#El valor de 'y' es la cantidad de alumnos desertores para el año 2011, multiplicando 2011 * .coef_[0], que en una ecuacion de recta sería igual al valor de la pendiente,
# mas el numero de semestre correspondiente multiplicado su coeficiente correspondiente y ademas el numero de id de carrera multiplicado por su coeficiente.
# Por ultimo y le sumo el .intercept_ que es el valor de la ordenada de origen
y_1=2011*(modelo.coef_[0]) + codigo*(modelo.coef_[1])+1*(modelo.coef_[2]) +modelo.intercept_
y_2=2011*(modelo.coef_[0]) + codigo*(modelo.coef_[1])+2*(modelo.coef_[2])+modelo.intercept_
#El siguiente codigo pone al valor de y_1 o y_2 como 0 si este es menor o igual a 0, ademas de redondear el valor a numero entero.
if y_1<=0:
  y_1=0
else:
  y_1=y_1.round()
if y_2<=0:
  y_2=0
else:
  y_2=y_2.round()
print(f'Se espera que en el año 2011 haya un total de {y_1.round()} Desertores en el primer semestre y {y_2.round()} para el segundo semestre, en la carrera con Id= {codigo}')

Ingrese el codigo de la carrera: 2
Se espera que en el año 2011 haya un total de 13.0 Desertores en el primer semestre y 32.0 para el segundo semestre, en la carrera con Id= 2


In [95]:
grupo_2

,Año,Id_Carrera,Semestre,cant
2,2001.0,2,1,18
3,2001.0,2,2,16
28,2002.0,2,1,5
29,2002.0,2,2,11
51,2003.0,2,1,6
52,2003.0,2,2,24
73,2004.0,2,1,5
74,2004.0,2,2,38
97,2005.0,2,1,5
98,2005.0,2,2,22


####Cantidad total de Matriculaciones esperadas en el año 2011

In [75]:
#Uno el DF df_Matriculacion con la tabla de carreras para así obtener el id de cada carrera
df_Matriculaciones_Carreras=df_Matriculacion.merge(df_Carreras)
df_Matriculaciones_Carreras.drop(['Cuatrimestres','Sexo','Rang_De_Edad_De_Ingreso_A_Carrera','Abreviatura','Orientacion','Nombre_Largo','Nombre_Corto'],
                                 axis=1,inplace=True)
df_Matriculaciones_Carreras

,Id_Alumno,Semestre_Ingreso,Año_Ingreso,Id_Carrera
0,0x0002EF737D1D58D4FE5B790A5290753F,2,2001,2
1,0x00765540B9CA6DBBE92820F203D443C6,1,2003,2
2,0x00C81052F138268CBE003F77E44E1FA6,1,2001,2
3,0x00E63AA4EA933EE138EF4587D5B8A351,2,2002,2
4,0x01593A043040D95BCF3E38092AAD61E0,1,2005,2
...,...,...,...,...
6199,0x4F8CDCC18445B66272EAD975E0DE6768,1,2005,39
6200,0x97DD39C55CA9AC960DD18887057C78A5,1,2005,39
6201,0xBBC26273A1E875EBA9696B72C1A6AC1D,1,2005,39
6202,0xEB424B36C0C3BD9BB5BE43C42F197B6D,1,2005,39


##### En la totalidad de la institución

In [96]:
#Genero un grupo del DF por año de ingreso contando la cantidad de alumnos que se inscribieron 
grupo_3=df_Matriculaciones_Carreras.groupby(['Año_Ingreso','Semestre_Ingreso'])['Id_Alumno'].count().reset_index(name='cant')
x=grupo_3.drop(['cant'],axis=1)
y=grupo_3['cant']
#Genero un modelo de Regresion Lineal, al ser pocos datos, no genero los sets de train y test
from sklearn import linear_model
modelo = linear_model.LinearRegression(fit_intercept=True)
modelo.fit(x,y)
#El valor de 'y' es la cantidad de alumnos desertores para el año 2011, multiplicando 2011 * .coef_[0], que en una ecuacion de recta sería igual al valor de la pendiente,
# mas el numero de semestre correspondiente * su coeficiente correspondiente y le sumo el .intercept_ que es el valor de la ordenada de origen
y_1=2011*(modelo.coef_[0]) + 1*(modelo.coef_[1])+modelo.intercept_
y_2=2011*(modelo.coef_[0]) + 2*(modelo.coef_[1])+modelo.intercept_
#El siguiente codigo pone al valor de y_1 o y_2 como 0 si este es menor o igual a 0, ademas de redondear el valor a numero entero.
if y_1<=0:
  y_1=0
else:
  y_1=y_1.round()
if y_2<=0:
  y_2=0
else:
  y_2=y_2.round()
print(f'Se espera que en el año 2011 haya un total de {y_1.round()} Matriculaciones en el primer semetre y {y_2.round()} para el segundo semestre')

Se espera que en el año 2011 haya un total de 578.0 Matriculaciones en el primer semetre y 132.0 para el segundo semestre


In [97]:
grupo_3

,Año_Ingreso,Semestre_Ingreso,cant
0,2001,1,583
1,2001,2,115
2,2002,1,420
3,2002,2,99
4,2003,1,338
5,2003,2,65
6,2004,1,457
7,2004,2,105
8,2005,1,551
9,2005,2,127


##### Por carrera

In [98]:
#El siguiente código es parecido al anterior, solo que se le añade un imput que genera el codigo de id de carrera para analizar datos específicos
grupo_4=df_Matriculaciones_Carreras.groupby(['Año_Ingreso','Id_Carrera','Semestre_Ingreso'])['Id_Alumno'].count().reset_index(name='cant')
codigo= int(input('Ingrese el codigo de la carrera: '))
grupo_4=grupo_4[grupo_4['Id_Carrera']==codigo]
x= grupo_4.drop(['cant'],axis=1)
y= grupo_4['cant']
#Genero un modelo de Regresion Lineal, al ser pocos datos, no genero los sets de train y test
from sklearn import linear_model
modelo = linear_model.LinearRegression(fit_intercept=True)
modelo.fit(x,y)
#El valor de 'y' es la cantidad de alumnos desertores para el año 2011, multiplicando 2011 * .coef_[0], que en una ecuacion de recta sería igual al valor de la pendiente,
# mas el numero de semestre correspondiente multiplicado su coeficiente correspondiente y ademas el numero de id de carrera multiplicado por su coeficiente.
# Por ultimo y le sumo el .intercept_ que es el valor de la ordenada de origen
y_1=2011*(modelo.coef_[0]) + codigo*(modelo.coef_[1])+1*(modelo.coef_[2])+modelo.intercept_
y_2=2011*(modelo.coef_[0]) + codigo*(modelo.coef_[1])+2*(modelo.coef_[2])+modelo.intercept_
#El siguiente codigo pone al valor de y_1 o y_2 como 0 si este es menor o igual a 0, ademas de redondear el valor a numero entero.
if y_1<=0:
  y_1=0
else:
  y_1=y_1.round()
if y_2<=0:
  y_2=0
else:
  y_2=y_2.round()
print(f'Se espera que en el año 2011 haya un total de {y_1} Matriculaciones en el primer semestre y {y_2} para el segundo semestre en la carrera con Id= {codigo}')

Ingrese el codigo de la carrera: 2
Se espera que en el año 2011 haya un total de 41.0 Matriculaciones en el primer semestre y 0 para el segundo semestre en la carrera con Id= 2


In [89]:
grupo_4

,Año_Ingreso,Id_Carrera,Semestre_Ingreso,cant
2,2001,2,1,91
3,2001,2,2,15
28,2002,2,1,69
29,2002,2,2,13
52,2003,2,1,45
53,2003,2,2,13
73,2004,2,1,55
74,2004,2,2,13
97,2005,2,1,60
98,2005,2,2,11
